# YouTube Comments Scrapping 
    
   **BY - AYUSH KUMAR MISHRA**

In [ ]:
#importing the required Libraries
import pandas as pd
import numpy as np
from selenium import webdriver 
import urllib
import pathlib
import time
from datetime import datetime

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--incognito')
driver = webdriver.Chrome( options=chrome_options)

search_item = ['Physical health']

Making a function for getting the links from the web

In [ ]:
# Function to get links
def get_links(item) :    
    query = urllib.parse.quote(item)
    url = "https://www.youtube.com/results?search_query=" + query
    driver.get(url)
    driver.execute_script("window.scrollTo(0, 1000000)")
    driver.maximize_window()
    container=driver.find_elements_by_xpath('//*[(@id = "video-title")]')
    links=[]
    for page in container:
        url=page.get_attribute("href")
        links.append(url)
    return list(set(links))

In [ ]:
dfs = []
for item in search_item:
    links = get_links(item)
    df = pd.DataFrame({'Item': [item] * len(links), 'Links': links})
    dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)  
final_df.to_csv('Links.csv', index=False)  
final_df

In [ ]:
len(final_df)

In [ ]:
null_counts = final_df.groupby('Item')['Links'].apply(lambda x: x.isnull().sum())
print(null_counts)

In [ ]:
df = final_df.dropna()
null_counts = df.groupby('Item')['Links'].apply(lambda x: x.isnull().sum())
print(null_counts)

In [ ]:
videos_dictionary = {}
videos_dictionary['Comments'] = {}
videos_dictionary['Video Link'] = {}

Defining the function to scrap the required information from the links

In [ ]:
def scrap(url,i):
    print('Fetched date and time - ',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    try :
        videos_dictionary['Video Link'].update({i : url})
    except:        
        videos_dictionary['Video Link'].update({i :''})
    driver.get(url)
    time.sleep(3)
    comments =  driver.find_element_by_xpath('//*[@id="comments"]')
    driver.execute_script("arguments[0].scrollIntoView();", comments)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    username_elems = driver.find_elements_by_xpath('//*[@id="author-text"]')
    comment_elems = driver.find_elements_by_xpath('//*[@id="content-text"]')
    comment_string = ''
    for user, comment in zip(username_elems, comment_elems):
        comment_string = comment_string + user.text + ' - ' + comment.text + '\n'
    try :
        videos_dictionary['Comments'].update({i : comment_string})
    except :
        videos_dictionary['Comments'].update({i : ''})

First check the file exist to save the data that has been scrapped from the links. If the file does not exist, then create one. If it exist then check whether it is empty or not.

If not empty, then extract data drom the links not present in the file.

In [ ]:
start = 0
name = 'Youtube_scrapping_comments.csv'
file_name = pathlib.Path(name)
if file_name.exists():
    print ("File exist.")
    try :
        print("Reading the file now")
        df_temp = pd.read_csv(name, index_col = 0)
        start = (len(df_temp) )
    except:
        print("File is empty")
else:
    print ("File does not exist\n", "Creating the file")
    file = open(name,"w+")
    print("File created successfully with filename - ",name)
print("Starting from position - ", start)

In [ ]:
# Scrapping the comments
for num in range(start, len(df)):
    url = df['Links'].iloc[num]  
    print("Loop entered")
    try:
        print("getting link-", num)
        driver.get(url)
    except:
        print("Not getting")
        continue
    print("=" * 40) 
    print("Scraping " + url)
    scrap(url, num)
    print("=" * 40) 
print("Process ended successfully")


In [ ]:
# Converting into table for better visuals
data = pd.DataFrame.from_dict(videos_dictionary)
try:
    result = pd.concat([df_temp,data], ignore_index = True)
    result
except :
    result = data
result

In [ ]:
results = result.dropna()
videos_dictionary = {}
results

In [ ]:
# results.iloc[[0]]['Comments']

# Print all comments for the first video link
for i in range(len(df)):
     vn = results.iloc[i]['Video Link']
     print(vn)
     print("=="*40)
     first_video_comments = results.iloc[i]['Comments']
     print(first_video_comments)
    

In [ ]:
driver.close()

# END

**Made By - AYUSH KUMAR MISHRA**